<h1 align=center><font size = 5>Segmenting and Clustering Neighbourhoods in Toronto</font></h1>

The notebook obtains Toronto neighborhoods data from https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M and 
converts addresses into their equivalent latitude and longitude values and generate maps to visualze neighbourhoods.

#### Import dependencies and libraries

In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

#### Download and Explore Dataset

scraping list of postal codes of canada

In [2]:
req = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")
soup = BeautifulSoup(req.content,'lxml')
table = soup.find_all('table')[0]
neighbourhood_df = pd.read_html(str(table))[0]

Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.

In [3]:
neighbourhood_df = neighbourhood_df[neighbourhood_df.Borough != 'Not assigned']

More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11 in the above table.

In [4]:
neighbourhood_df['Neighbourhood'] = neighbourhood_df.groupby('Postal Code')['Neighbourhood'].transform(lambda x: ','.join(x))
neighbourhood_df.drop_duplicates()
neighbourhood_df = neighbourhood_df.reset_index().drop(['index'], axis=1)

In [5]:
neighbourhood_df.head(15)

,Postal Code,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
6,M1B,Scarborough,"Malvern, Rouge"
7,M3B,North York,Don Mills
8,M4B,East York,"Parkview Hill, Woodbine Gardens"
9,M5B,Downtown Toronto,"Garden District, Ryerson"


In [6]:
print('The dataframe has {} boroughs and {} neighbourhoods.'.format(
        len(neighbourhood_df['Borough'].unique()),
        neighbourhood_df.shape[0]
    )
)

The dataframe has 10 boroughs and 103 neighbourhoods.


Since geocoder is not reliable, geographical coordinates are loaded from 'http://cocl.us/Geospatial_data'. Then merge coordinates data with postcode_df.

In [7]:
geospatial_df = pd.read_csv('http://cocl.us/Geospatial_data')
neighbourhood_df = neighbourhood_df.merge(geospatial_df, left_on='Postal Code', right_on='Postal Code')

In [8]:
neighbourhood_df.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


####  Use geopy library to get the latitude and longitude values of Toronto.

In [9]:
address = 'Toronto, Ontario'

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


#### Get boroughs that contain the word Toronto only

In [10]:
toronto_neighbourhood = neighbourhood_df[neighbourhood_df['Borough'].str.contains('Toronto')]
toronto_neighbourhood.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
15,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
19,M4E,East Toronto,The Beaches,43.676357,-79.293031


#### Create a map of Toronto with neighborhoods superimposed on top.

In [11]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighbourhood in zip(toronto_neighbourhood['Latitude'], toronto_neighbourhood['Longitude'], toronto_neighbourhood['Borough'], toronto_neighbourhood['Neighbourhood']):
    label = '{}, {}'.format(neighbourhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto